In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, time

import warnings
warnings.filterwarnings('ignore')

In [10]:
starting_dir = os.getcwd()
print (starting_dir)

/Users/kimd999/research/script_not_in_dropbox/srpAnalytics/analysis/latest/1_reformat/all_targets/LPR


### Reformat LPR behavioral data

In [3]:
# mac - phase I & II - full
#complete_input_file_path= '/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/LPR/tall/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall.csv'

# mac - phase I & II - 240 endpoints
complete_input_file_path= '/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/LPR/tall/bifurcated/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall_w_240_endpoints.csv'

# mac - phase I & II - 15 endpoints
#complete_input_file_path= '/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/LPR/tall/bifurcated/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall_w_15_endpoints.csv'

# mac - phase III - full
#complete_input_file_path= '/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/original/behavior/LPR/Tanguay_Phase_3_zf_LPR_data_PNNL_2021MAR23.csv'


# constance - phase I & II - full
#complete_input_file_path= '/qfs/people/kimd999/tox/phase_I_II/LPR/input/tall/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall_full_w_15_endpoints.csv'
#complete_input_file_path= '/qfs/people/kimd999/tox/phase_I_II/LPR/input/tall/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall_full_w_240_endpoints.csv'

# constance - phase III - full
#complete_input_file_path= '/people/kimd999/tox/phase_III/Tanguay_Phase_3_zf_LPR_data_PNNL_2021MAR23.csv'

In [11]:
behav_all_data = pd.read_csv(complete_input_file_path, header = 0)
behav_all_data = behav_all_data.rename({'endpoint': 'variable'}, axis=1)
display(behav_all_data.head())

,chemical.id,conc,plate.id,well,variable,value
0,53,NaN,NaN,NaN,t0,NaN
1,53,0.0,9289.0,A06,t0,0.0
2,53,0.0,9289.0,A12,t0,0.0
3,53,0.0,9289.0,B06,t0,0.0
4,53,0.0,9289.0,B12,t0,0.0


In [12]:
# Keep only relevant columns
columns_to_keep = ['chemical.id', 'conc', 'plate.id', 'well', 'variable', 'value']
behav_select = behav_all_data.loc[:,columns_to_keep]

display("number of unique variable:" + str(len(np.unique(behav_select['variable']))))
display("number of chemical_id:" + str(len(np.unique(behav_select['chemical.id']))))
# phase I && II
# 240 variables -> 196 chemicals
# 15  variables -> 148 chemicals

# phase III
# 240 variables -> 215 chemicals

'number of unique variable:240'

'number of chemical_id:196'

In [13]:
nan = behav_select[behav_select['value'].isna()]
display(nan.head())

# [phase III] there is no nan in 'chemical.id', 'conc', 'plate.id', 'well', 'variable'

,chemical.id,conc,plate.id,well,variable,value
0,53,NaN,NaN,NaN,t0,NaN
193,53,NaN,NaN,NaN,t1,NaN
386,53,NaN,NaN,NaN,t2,NaN
579,53,NaN,NaN,NaN,t3,NaN
772,53,NaN,NaN,NaN,t4,NaN


In [14]:
display("before dropna, len(behav_select):"+str(len(behav_select)))
behav_select = behav_select.dropna(how='any')
display("after dropna,  len(behav_select):"+str(len(behav_select)))

display(behav_select.head())

# phase I & II -> dropped some
# phase III    -> dropped many

'before dropna, len(behav_select):11184240'

'after dropna,  len(behav_select):10977439'

,chemical.id,conc,plate.id,well,variable,value
1,53,0.0,9289.0,A06,t0,0.0
2,53,0.0,9289.0,A12,t0,0.0
3,53,0.0,9289.0,B06,t0,0.0
4,53,0.0,9289.0,B12,t0,0.0
5,53,0.0,9289.0,C06,t0,1.0


In [ ]:
''' # basic check of variable #
full_devel = "full"
#full_devel = "devel"

if (full_devel == "full"):
    chemical_id_from_here = np.unique(behav_select['chemical.id'])
else: # full_devel = "devel"
    chemical_id_from_here = np.unique([234])
    
for chemical_index in chemical_id_from_here:
    behav_per_chemical = behav_select.loc[behav_select['chemical.id'] == chemical_index,:]
    print("chemical_index:" + str(chemical_index))

    variables = np.unique(behav_per_chemical['variable'])
    print("variables:" + str(variables))
#    variable_splited = variable.split("t")
    
 #   print (min(variable_splited[1]))
    var_len = len(np.unique(behav_per_chemical['variable']))
    if (var_len != 15):
        display("number of variable:" + str(len(np.unique(behav_per_chemical['variable']))))
'''

### Transpose time points 

In [ ]:
start_time = time.time()
       
def reformat(chemical_index, behav_select, reformatted):
    behav_per_chemical = behav_select.loc[behav_select['chemical.id'] == chemical_index,:]
    #print (behav_per_chemical)

    # Append chemical_plate_well as a unique identifier
    # takes long time (1 min?)
    behav_per_chemical.insert(0, 'chemical_plate_well', behav_per_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))
    
    for cpw in np.unique(behav_per_chemical.chemical_plate_well):
        #print (str(cpw))
        per_cpw = behav_per_chemical.loc[behav_per_chemical.chemical_plate_well == cpw,:]
        per_cpw_grouped = per_cpw.groupby(['chemical.id', 'plate.id', 'well'])
        for name, group in per_cpw_grouped:
            final = pd.DataFrame(
                    {
                    'chemical.id': np.unique(per_cpw['chemical.id']),
                    'plate.id': np.unique(per_cpw['plate.id']),
                    'well': np.unique(per_cpw['well']),
                    'chemical_plate_well': np.unique(per_cpw['chemical_plate_well']),
                    'conc': np.unique(per_cpw['conc'])
                    })
            # Append additonal columns corresponding to time points
            for time_point in np.arange(len(np.unique(group.variable))):
                #print ("np.unique(group.variable):"+str(np.unique(group.variable)))
                if (len(np.unique(group.variable)) == 15):
                    time_point = time_point + 3
                variable = 't'+ str(time_point)
                #print ("\nvariable:"+str(variable))
                final = pd.concat([final, pd.DataFrame({variable: per_cpw.value[per_cpw.variable == variable].values})],axis = 1)
            reformatted = pd.concat([reformatted, final])
    return reformatted
########### end of def reformat(chemical_index, behav_select, reformatted):

reformatted = pd.DataFrame()

#full_devel = "full"
full_devel = "devel"

if (full_devel == "full"):
    chemical_id_from_here = np.unique(behav_all_data['chemical.id'])
else: # full_devel = "devel"
    #chemical_id_from_here = [53,54]
    chemical_id_from_here = [414]

for chemical_index in chemical_id_from_here:
    print("chemical_index:" + str(chemical_index))
    reformatted = reformat(chemical_index, behav_select, reformatted)

end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("Done, it took:"+str(time_took)) 
# took 2 minutes for 2 chemicals

chemical_index:414
np.unique(group.variable):['t0' 't1' 't10' 't100' 't101' 't102' 't103' 't104' 't105' 't106' 't108'
 't109' 't11' 't110' 't111' 't112' 't114' 't115' 't116' 't117' 't118'
 't119' 't12' 't120' 't121' 't123' 't124' 't125' 't126' 't128' 't129'
 't13' 't130' 't131' 't133' 't134' 't135' 't136' 't137' 't138' 't139'
 't14' 't140' 't141' 't142' 't143' 't144' 't145' 't146' 't147' 't148'
 't149' 't15' 't150' 't151' 't152' 't153' 't154' 't156' 't157' 't158'
 't16' 't160' 't161' 't162' 't163' 't164' 't165' 't166' 't167' 't169'
 't17' 't170' 't171' 't172' 't173' 't174' 't175' 't176' 't178' 't179'
 't18' 't180' 't181' 't182' 't183' 't184' 't185' 't186' 't187' 't188'
 't189' 't19' 't190' 't191' 't192' 't193' 't194' 't195' 't196' 't197'
 't198' 't199' 't2' 't20' 't200' 't201' 't202' 't203' 't204' 't205' 't206'
 't207' 't209' 't21' 't210' 't211' 't212' 't213' 't214' 't215' 't216'
 't217' 't218' 't219' 't22' 't220' 't221' 't222' 't223' 't224' 't225'
 't226' 't227' 't228' 't229' 't23' 't

In [ ]:
# ds = pd.Series({"Column": len_group_variable})
# plt.figure(figsize=(8,4))
# sns.countplot(x="Column", data=ds)
# plt.show()

In [51]:
display(reformatted.head())

,chemical.id,plate.id,well,chemical_plate_well,conc,t0,t1,t2,t3,t4,...,t218,t219,t220,t221,t222,t223,t224,t225,t226,t227
0,414,10184.0,A01,414_10184.0_A01,50.0,6.0,0.1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.3,2.0,0.0,0.0,0.0
0,414,10184.0,A02,414_10184.0_A02,35.6,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,29.9,0.4,0.0,0.0,2.4,0.0
0,414,10184.0,A03,414_10184.0_A03,11.2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,414,10184.0,A04,414_10184.0_A04,5.0,10.1,0.0,0.0,0.0,18.0,...,0.0,0.0,2.8,12.3,27.5,6.3,4.3,0.0,0.0,0.0
0,414,10184.0,A05,414_10184.0_A05,1.0,6.6,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
reformatted_data_filename = str(complete_input_file_path[:-9]) + "_wide_t0_t239_" + str(full_devel) + "_414.csv"
display ("reformatted_data_filename:", reformatted_data_filename)
reformatted.to_csv(reformatted_data_filename,index=False)

# 15 variables
#reformatted_data_filename = str(complete_input_file_path[:-9]) + "_wide_t0_t239_" + str(full_devel) + "_w_non_240_endpoints.csv"
#reformatted_w_non_240_endpoints.to_csv(reformatted_data_filename,index=False)

'reformatted_data_filename:'

'/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/LPR/tall/bifurcated/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall_w_240_endp_wide_t0_t239_devel_414.csv'

In [40]:
display(len(np.unique(reformatted['chemical.id'])))
#display(len(np.unique(reformatted_w_non_240_endpoints['chemical.id'])))

1